# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from googlekey import googlekey

# Configure gmaps
gmaps.configure(api_key=googlekey)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind,Weather Description
0,0,Ushuaia,AR,-54.8000,-68.3000,35.26,87,20,17.27,few clouds
1,1,Shache,CN,38.4167,77.2406,65.73,20,99,3.29,overcast clouds
2,2,Rikitea,PF,-23.1203,-134.9692,70.97,72,100,18.86,overcast clouds
3,3,Concepcion,CL,-36.8270,-73.0498,51.60,82,100,6.91,overcast clouds
4,4,Puerto Ayora,EC,-0.7393,-90.3518,66.90,81,64,9.57,broken clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [3]:
# Prompt the user to enter the minimum temperature criteria
min_temp =float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the max temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the max temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind,Weather Description
6,6,Sao Miguel Do Araguaia,BR,-13.2750,-50.1628,76.98,39,3,0.76,clear sky
9,9,Poum,NC,-20.2333,164.0167,76.32,74,52,12.66,broken clouds
10,10,Itarema,BR,-2.9248,-39.9167,78.01,81,14,17.40,few clouds
18,18,Atuona,PF,-9.8000,-139.0333,76.98,74,7,14.72,clear sky
29,29,Mahebourg,MU,-20.4081,57.7000,77.29,61,40,9.22,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
653,653,Diamantino,BR,-14.4086,-56.4461,76.55,65,55,2.93,broken clouds
654,654,Ixtapa,MX,20.7000,-105.2000,80.22,83,100,2.48,overcast clouds
657,657,Matara,LK,5.9485,80.5353,82.54,74,100,13.69,overcast clouds
658,658,Adrar,MR,20.5022,-10.0711,80.98,16,64,7.40,broken clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [5]:
preferred_cities_df.isnull().sum() #no nulls detected
preferred_cities_df.dropna() #dropping just incase
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df[["City", "Country", "Max Temp", "Lat",
                                "Lng", "Max Temp", "Humidity","Cloudiness","Wind",
                                          "Weather Description"]].copy()

# Display sample data
clean_travel_cities.dtypes

City                    object
Country                 object
Max Temp               float64
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind                   float64
Weather Description     object
dtype: object

## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [6]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat",
                                "Lng","Weather Description"]].copy()
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Weather Description
6,Sao Miguel Do Araguaia,BR,76.98,-13.2750,-50.1628,clear sky
9,Poum,NC,76.32,-20.2333,164.0167,broken clouds
10,Itarema,BR,78.01,-2.9248,-39.9167,few clouds
18,Atuona,PF,76.98,-9.8000,-139.0333,clear sky
29,Mahebourg,MU,77.29,-20.4081,57.7000,scattered clouds


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [7]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(5)

,City,Country,Max Temp,Lat,Lng,Weather Description,Hotel Name
6,Sao Miguel Do Araguaia,BR,76.98,-13.2750,-50.1628,clear sky,
9,Poum,NC,76.32,-20.2333,164.0167,broken clouds,
10,Itarema,BR,78.01,-2.9248,-39.9167,few clouds,
18,Atuona,PF,76.98,-9.8000,-139.0333,clear sky,
29,Mahebourg,MU,77.29,-20.4081,57.7000,scattered clouds,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [8]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": googlekey
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [9]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Weather Description,Hotel Name
6,Sao Miguel Do Araguaia,BR,76.98,-13.2750,-50.1628,clear sky,Hotel Executivo Palace
9,Poum,NC,76.32,-20.2333,164.0167,broken clouds,
10,Itarema,BR,78.01,-2.9248,-39.9167,few clouds,"Sea Lounge! Bar, Club & Restaurant"
18,Atuona,PF,76.98,-9.8000,-139.0333,clear sky,Villa Enata
29,Mahebourg,MU,77.29,-20.4081,57.7000,scattered clouds,Shandrani Beachcomber Resort & Spa


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [11]:
import numpy as np #importing numpy to use np.nan
hotel_df["Hotel Name"].replace("",np.nan,inplace= True) #setting blank strings to nans for dropna
hotel_df.dropna(inplace = True)
clean_hotel_df = hotel_df.copy()

# Display sample data
clean_hotel_df


,City,Country,Max Temp,Lat,Lng,Weather Description,Hotel Name
6,Sao Miguel Do Araguaia,BR,76.98,-13.2750,-50.1628,clear sky,Hotel Executivo Palace
10,Itarema,BR,78.01,-2.9248,-39.9167,few clouds,"Sea Lounge! Bar, Club & Restaurant"
18,Atuona,PF,76.98,-9.8000,-139.0333,clear sky,Villa Enata
29,Mahebourg,MU,77.29,-20.4081,57.7000,scattered clouds,Shandrani Beachcomber Resort & Spa
32,Hilo,US,77.29,19.7297,-155.0900,overcast clouds,Hilo Hawaiian Hotel
...,...,...,...,...,...,...,...
648,Galle,LK,80.01,6.0367,80.2170,overcast clouds,Amangalla
653,Diamantino,BR,76.55,-14.4086,-56.4461,broken clouds,Hotel Delta
654,Ixtapa,MX,80.22,20.7000,-105.2000,overcast clouds,hotel la terminal
657,Matara,LK,82.54,5.9485,80.5353,overcast clouds,Amaloh Boutique Resort


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [12]:
#In case of an empty DataFrame, load the sample data provided
#clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
#clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [13]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file)

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [14]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [15]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[10]

'\n<dl>\n<dt>Hotel Name</dt><dd>Fengshui Residence</dd>\n<dt>City</dt><dd>Port-Gentil</dd>\n<dt>Country</dt><dd>GA</dd>\n<dt>Current Weather</dt><dd>thunderstorm with light rain and 76.96 °F</dd>\n</dl>\n'

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [16]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
6,-13.2750,-50.1628
10,-2.9248,-39.9167
18,-9.8000,-139.0333
29,-20.4081,57.7000
32,19.7297,-155.0900
37,-20.5167,57.5167
38,-6.6024,147.8681
40,-2.8833,108.2667
55,25.3833,97.4000
62,-5.6667,132.7500


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [17]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Create a figure to add the Google map as a layer
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))